## Install libraries if not installed 

In [1]:
# Uncomment below lines to install

# pip install earthengine-api
# pip install geemap
# pip install folium

## Import all the libraries

In [2]:
# Earth engine needs authentication. Once authenticated, user can use the same token for a week.
# But it should be initialized every time the user runs the code

import ee
ee.Initialize()
# ee.Authenticate()

In [3]:
import geemap

# Sent2DownProcNDVI class is imported from sent2ndvi and an object is created
from sent2ndvi import Sent2DownProcNDVI
s2dpn = Sent2DownProcNDVI()

## Load the shapefile

In [4]:
# load_shape_file function is accessed from the sent2ndvi module to load a shapefile and the shapefile is then
# converted to a geometry

vskp_city = s2dpn.load_shape_file('/Users/saimanojappalla/Desktop/SatSureAssignment/district shapefile/vskp_city.shp')
vskp_geom = vskp_city.geometry()

## Initialize all the parameters

In [5]:
# The attributes inside the Sent2DownProcNDVI class in sent2ndvi module are accessed using the object and are 
# assigned values by the user

s2dpn.AOI = ee.Geometry.Point(83.2185, 17.6868)
s2dpn.START_DATE = '2022-05-01'
s2dpn.END_DATE = '2022-09-01'
s2dpn.CLOUD_FILTER = 60
s2dpn.CLD_PRB_THRESH = 40
s2dpn.NIR_DRK_THRESH = 0.15
s2dpn.CLD_PRJ_DIST = 2
s2dpn.BUFFER = 100

## Create a collection with Sentinel-2 surface reflectance and Cloud probability data

In [6]:
# This line of code imports the get_s2_sr_cld_col function from the sent2ndvi module and returns the image collection
# based on the parameters input by the user

coll = s2dpn.get_collection(s2dpn.AOI, s2dpn.START_DATE, s2dpn.END_DATE)


## Create a composite image by mapping the functions to apply cloud mask

In [7]:
# This line of code imports the functions from the sent2ndvi module and returns the composite image after masking
# the clouds

s2_sr_median = s2dpn.cloud_masked_composite(coll)

## Use Folium library to view the image on the map 

In [8]:
# Import the folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [9]:
# Create a folium map object.
center = s2dpn.AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12)

# Add layers to the folium map.
m.add_ee_layer(s2_sr_median,
                {'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free mosaic', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

## NDVI

In [10]:
# This line of code imports the ndvi function from the sent2ndvi module and returns the ndvi image

ndvi = s2dpn.ndvi(s2_sr_median)

## Download

In [11]:
# This line of code imports the download function from the sent2ndvi module and downloads the image

s2dpn.download(ndvi, vskp_geom, 30, 'ndvi.tif')

Generating URL ...
Please wait ...
Data downloaded to /Users/saimanojappalla/Desktop/SatSureAssignment/ndvi.tif
